# 02-流式输出 - DeepSeek API

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# 加载 .env 文件
load_dotenv(dotenv_path='../../.env')  # 根据实际路径调整

# 从环境变量获取 API Key
api_key = os.getenv("VITE_DEEPSEEK_API_KEY") or os.getenv("DEEPSEEK_API_KEY")
if not api_key:
    raise ValueError("❌ 未找到 API Key，请在 .env 文件中设置 VITE_DEEPSEEK_API_KEY")

base_url = os.getenv("VITE_DEEPSEEK_BASE_URL", "https://api.deepseek.com")

# 初始化客户端
client = OpenAI(
    api_key=api_key,
    base_url=base_url,
)

print("✅ DeepSeek 客户端初始化成功")
print(f"📍 Base URL: {base_url}")

✅ DeepSeek 客户端初始化成功
📍 Base URL: https://api.deepseek.com


## 基础流式输出

In [2]:
# 基础流式输出
response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[{"role": "user", "content": "讲一个短故事"}],
    stream=True
)

print("📝 流式输出：")
for chunk in response:
    if chunk.choices[0].delta.content:
        print(chunk.choices[0].delta.content, end="", flush=True)

print()

📝 流式输出：
## 我在地府当社畜
>阎王说我阳寿未尽，但生死簿上名字已勾。
>他挠头：“要不你先在地府打份工？等我们查清楚。”
>于是我成了地府临时工，每天帮孟婆研发新口味汤品。
>直到那天，我无意瞥见生死簿——我的名字旁，赫然签着阎王的私印。
>原来我的死，是他亲手批的。

---

判官殿里那点青幽幽的鬼火，跳得人心慌。我杵在堂下，骨头缝里都往外冒凉气。上头那位，黑脸膛，络腮胡，戴着一顶歪歪扭扭的冕旒，正把一本巨大的册子翻得哗啦响，眉头拧得能夹死苍蝇。

“李小明？”他瓮声瓮气，抬起眼皮瞥我。

“是……是我。”我舌头有点打结。上一秒还在电脑前敲方案，下一秒眼前一黑，再睁眼就到这鬼地方了。路上那个穿黑西装、自称“阴差”的哥们儿倒是客气，一路“这边请”、“小心台阶”，可越客气我腿越软。

“怪事……”阎王——我猜他是——挠了挠他那头乱发，冕旒上的珠子跟着乱晃，“阳寿显示未尽，还有……我看看，四十三年零七个月十八天。可这生死簿上……”他把那本厚得能砸死人的册子转过来，指向其中一行。

我的名字。后面跟着生辰八字。再后面，一个鲜红的、刺眼的勾。

“名字已经勾了。”阎王摊手，一脸“这事儿整的”的无奈，“按规矩，勾了就得留下。可你这阳寿明明没完……系统bug？不对，生死簿是天道规则显化，哪来的bug……”

他嘀嘀咕咕，又把册子翻回去看，手指头一行行往下捋，嘴里念念有词。殿侧侍立着几个青面鬼差，眼观鼻鼻观心，泥塑木雕一般。只有鬼火偶尔“噼啪”一声，炸开一点惨绿的光。

我心脏像被一只冰冷的手攥住了，越收越紧。留……留下？开什么玩笑！我方案还没交！房租还没交！我……

“有了！”阎王忽然一拍大腿，吓我一跳。他眼睛亮了亮，看向我，那眼神活像程序员逮住个能测试新代码的小白鼠，“这样，反正一时半会儿也查不清缘由。地府最近嘛……业务扩张，也讲个与时俱进，正缺人手。你呢，就先在这儿干着，算……嗯，临时工！包吃包住，没有五险一金，但阴德可以攒一点，等我们查明原因，该送你回去就送你回去，怎么样？”

我能说不怎么样吗？看看旁边那几位鬼差大哥腰间若隐若现的锁链，我咽了口唾沫，把冲到喉咙口的抗议压了回去。“……干什么活？”

“放心，轻松活儿！”阎王笑得有点过于爽朗，让人心里发毛，“奈何桥头，孟婆那儿，正缺个有创新精神的帮手。你去正合适！”

于是，我的地府职业生涯，就从奈何桥头那口咕

## 思考模型流式输出

In [ ]:
# deepseek-reasoner 流式输出（含思考过程）
response = client.chat.completions.create(
    model="deepseek-reasoner",
    messages=[{"role": "user", "content": "证明勾股定理"}],
    stream=True,
    max_tokens=4096
)

print("🧠 思考过程：")
print("-" * 60)

in_content = False
reasoning_parts = []
content_parts = []

for chunk in response:
    delta = chunk.choices[0].delta
    
    # 获取思考过程
    reasoning = getattr(delta, "reasoning_content", None)
    if reasoning:
        reasoning_parts.append(reasoning)
        print(reasoning, end="", flush=True)
    
    # 正式答案在思考过程之后
    if delta.content:
        if not in_content:
            in_content = True
            print("\n\n📄 正式答案：")
            print("-" * 60)
        content_parts.append(delta.content)
        print(delta.content, end="", flush=True)

print("\n")
print("=" * 60)
print(f"思考部分: {len(''.join(reasoning_parts))} 字符")
print(f"回答部分: {len(''.join(content_parts))} 字符")

🧠 思考过程：
------------------------------------------------------------
我们被要求证明勾股定理。勾股定理指出，在一个直角三角形中，两条直角边的平方和等于斜边的平方。即，如果直角三角形的两条直角边长为a和b，斜边长为c，那么a² + b² = c²。

有许多方法可以证明勾股定理。我将选择一种经典的证明，可能是欧几里得的证明，或者是一种简单直观的证明。考虑到这是一个数学问题，我应该提供一个清晰、逻辑严谨的证明。

我将使用一种基于相似三角形的证明，或者使用面积法。一种常见的直观证明是通过构造一个正方形，其边长为a+b，然后比较面积。

让我回忆一下：构造一个边长为a+b的正方形，内部以四种方式放置四个全等的直角三角形（直角边为a和b，斜边为c）。这样，中间形成一个边长为c的正方形。然后计算大正方形的面积有两种方法：一是直接计算(a+b)²，二是计算四个三角形的面积加上中间正方形的面积。即：(a+b)² = 4*(1/2 * a * b) + c²。化简得到 a²+2ab+b² = 2ab + c²，所以 a²+b² = c²。

这是最流行的证明之一，通常称为“毕达哥拉斯定理的证明”或“面积证明”。我将详细描述这个证明。

证明如下：

考虑一个直角三角形，其直角边长为a和b，斜边长为c。以a+b为边长构造一个正方形。在这个正方形内部，以如下方式放置四个全等的直角三角形：每个三角形的直角边与正方形的边重合，使得每个三角形的斜边构成一个较小的正方形。具体来说，将四个直角三角形分别放置在大正方形的四个角上，使得每条直角边分别沿着大正方形的边。这样，四个直角三角形的斜边会围成一个四边形。由于四个三角形全等，这个四边形的四条边相等，且每个角都是直角（因为每个三角形的两个锐角互补，可以证明围成的四边形的角是90度）。因此，中间围成的四边形是一个边长为c的正方形。

现在，计算整个大正方形的面积。大正方形的边长为a+b，所以面积为 (a+b)²。

另一方面，大正方形的面积等于四个直角三角形的面积加上中间小正方形的面积。每个直角三角形的面积为 (1/2)ab，四个三角形的总面积为 4*(1/2)ab = 2ab。中间小正方形的边长为c，面积为 c²。因此，总面积也为 2ab + c²。

所以，(a+b)² = 2

## 完整流式处理类

In [ ]:
class DeepSeekStreamingChat:
    """DeepSeek 流式聊天封装"""
    
    def __init__(self, client):
        self.client = client
    
    def chat(self, message, model="deepseek-chat", show_reasoning=True, **kwargs):
        """流式对话"""
        response = self.client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": message}],
            stream=True,
            **kwargs
        )
        
        full_content = []
        reasoning_content = []
        in_content = False
        
        for chunk in response:
            delta = chunk.choices[0].delta
            
            # 获取思考内容
            reasoning = getattr(delta, "reasoning_content", None)
            if reasoning:
                if show_reasoning and not reasoning_content:
                    print("\033[90m", end="")  # 灰色
                reasoning_content.append(reasoning)
                if show_reasoning:
                    print(reasoning, end="", flush=True)
            
            # 获取正式内容
            if delta.content:
                if not in_content:
                    in_content = True
                    if show_reasoning and reasoning_content:
                        print("\033[0m")  # 恢复颜色
                        print("\n")
                full_content.append(delta.content)
                print(delta.content, end="", flush=True)
        
        print()
        
        return {
            "content": "".join(full_content),
            "reasoning": "".join(reasoning_content) if reasoning_content else None
        }

# 使用
chat = DeepSeekStreamingChat(client)
result = chat.chat("解释量子力学", model="deepseek-reasoner")
print(f"\n总结: 思考 {len(result['reasoning'] or '')} 字符, 回答 {len(result['content'])} 字符")

量子力学是描述微观粒子行为的物理学理论，涉及到波函数、概率幅、算符、测量等核心概念。用户的问题是“解释量子力学”，这是一个非常广泛的话题，需要选择合适的深度和广度进行回答。考虑到用户没有指定具体的细节，应该提供一个全面但简洁的概述，覆盖基本原理、关键概念和历史背景，同时避免过于专业的数学细节。

用户可能是一个对量子力学感兴趣的非专业人士，或者是一个学生，希望获得一个入门级的解释。因此，回答应该易于理解，使用类比和直观描述，同时保持准确性。重点应该放在量子力学与经典物理的区别，以及它如何改变我们对世界的认识。

回答的结构可以包括：引言、核心概念（如波粒二象性、不确定性原理、叠加态、纠缠）、数学框架简要说明、测量问题、以及量子力学的应用和意义。最后可以提及一些未解之谜和当前的研究方向，以激发兴趣。


量子力学是描述微观粒子（如原子、电子、光子等）行为的物理学理论，它与相对论共同构成了现代物理学的基石。以下是其核心概念的简明解释：

---

### **1. 量子力学的诞生背景**
- **经典物理的局限**：19世纪末，牛顿力学和电磁理论无法解释黑体辐射、原子光谱等实验现象。
- **量子假设的提出**：1900年，普朗克提出能量量子化假设，解释黑体辐射；1905年，爱因斯坦提出光量子（光子）概念，解释光电效应。

---

### **2. 核心原理**
#### **（1）波粒二象性**
- 微观粒子既表现粒子性（如能量、动量集中），又表现波动性（干涉、衍射）。
- **德布罗意关系**：所有物质都具有波长，\(\lambda = \frac{h}{p}\)（\(h\)为普朗克常数，\(p\)为动量）。

#### **（2）量子态与波函数**
- 粒子状态由**波函数** \(\Psi(x,t)\) 描述，其模的平方 \(|\Psi|^2\) 表示粒子在空间某点出现的**概率密度**。
- 波函数遵循**薛定谔方程**：\(i\hbar \frac{\partial \Psi}{\partial t} = \hat{H} \Psi\)，决定量子态随时间演化。

#### **（3）不确定性原理**
- 海森堡提出：无法同时精确测量粒子的位置和动量，\(\Delta x \Delta p \geq \frac{\hbar}{2}\)。
- 本质是波函数的内在